# case #1 : real, pair-wise different eigenvalues of diagonalizable $A$

- $\sigma(A) = [\lambda_1, \ldots, \lambda_n]$
- $\lambda_1 > \lambda_2 > \ldots > \lambda_n \in \mathbb{R}$
- assume two "views" onto the latent system via $A, \tilde{A}$, $Q, \tilde{Q}$ etc.
- without loss of generality , assume $\tilde{A} = \Lambda = \mbox{diag}(\lambda_1, \ldots, \lambda_n)$

What can we learn from the latent state matrices $A, \tilde{A}$, and $Q, \tilde{Q}$ alone, i.e. not looking at $C, \tilde{C}$?

- what we have: $A, \Lambda$ are similar ($A=M \Lambda M^{-1}$), $Q$ and $\tilde{Q}$ are linked via the same $M$ (through $Q = M \tilde{Q} M^\top$)

Results: 

- $A=M \Lambda M^{-1} = WS\Lambda (WS) ^{-1}$ allows identifying $M=WS$ only up to a matrix $S \in \mathbb{R}^{n \times{} n}$
- **restrictions** on $S$ are key now. Seemingly, $S = \mbox{diag}(s_1, \ldots, s_n)$ is diagonal - **not entirely sure yet why**
- not knowing the per-latent scales $s_i \in \mathbb{R}$ is even worse than only not knowing the correct sign   
- latent covariances $Q, \tilde{Q}$ resolve the scales $|s_i|$ 
- $Q, \tilde{Q}$  also resolve the signs $\mbox{sign}(s_i)$ assuming they are not diagonal  

The code belows serves to play out those conditions numerically 

In [ ]:
import numpy as np
from scipy import stats as stats

p, n = 5, 4

M = np.random.normal(size=(n,n)) # change of basis (actually only has to be invertible)

# system matrices in first (=standard) coordinate system
D = np.sort(np.random.normal(size=n))        # draw real, pairwise different eigenvalues 
L = np.diag(D)
Q = stats.wishart.rvs(df=n, scale=np.eye(n)) 
C = np.random.normal(size=(p,n))

# system matrices in second coordinate system
Ar = M.dot(L).dot(np.linalg.inv(M))
Qr = M.dot(Q).dot(M.T)
Cr = C.dot(np.linalg.inv(M))

# recovering original matrices by explicity rotations using established conditions
Lh, Wh = np.linalg.eig(Ar)
idx = np.argsort(Lh)  
Lh, Wh = Lh[idx], Wh[:, idx]

Winv = np.linalg.inv(Wh)
SQS = Winv.dot(Qr).dot(Winv.T)
s = np.sqrt( np.diag(SQS) / np.diag(Q) ) 

signs = (s*Q*s.reshape(-1,1)) / SQS # all pairwise signs, simple reconstruction approach

s *= signs[0,:] # lazy, just align all signs according to first variable

print('compare each entry of real nxn base-change matrix M with reconstruction W*S via M /(W*S) : ')
print(M / Wh.dot(np.diag(s)))